In [2]:
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; float:center}</style>")

In [3]:
from __future__ import division, print_function

import os
import glob
import math
import re
from collections import Counter
import datetime
from itertools import groupby

import pandas as pd
import numpy as np
import sklearn as skl
# import scipy as sp
# import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud as wc


import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
# from nltk import Text

import time

In [4]:
porter = PorterStemmer()

%matplotlib inline

sns.set_context('notebook')
sns.set_style('ticks')

In [5]:
stops = set(stopwords.words('english'))
    # using a set will make it faster to run through...

punctuation = ['.',',',':','!',';','-','?','"',"'",'(',')','—']   
other = ['deb','hideb','don','didn','twaittry','doesn','thank','heydeb','ve', "i've", "i'v"]
    
stops_punc = set(stopwords.words('english') + punctuation)

mystops = stopwords.words('english') + punctuation + other
mystops_set = set(stopwords.words('english') + punctuation + other)

import & sanity check

In [8]:
comments = pd.read_csv('/Users/kateliea/Documents/Insight/project/data/comments_intermediateSKdata_annotated.csv', index_col=0)
# recipes = pd.read_csv('/Users/kateliea/Documents/Insight/project/data/comments_intermediateSKdata_annotated.csv', index_col=0)

len(comments), comments.columns #, len(recipes) #,recipes.numberofcomments.sum()

/Users/kateliea/anaconda/envs/py3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(65499, Index(['category', 'sentence', 'commentID', 'child_id', 'children',
        'comment_time', 'recipenumber', 'title', 'url', 'usercomment',
        'username', 'usersite', 'usercomment_lower', 'tokens',
        'tokens_stemmed'],
       dtype='object'))

In [48]:
comments.loc[486,'category'] = 'suggestion'

In [54]:
comments_classified.to_csv('comments_classified_SK_filtered2000.csv')

## word relevancy - tf-idf

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count = CountVectorizer()
tfidf = TfidfTransformer()

In [ ]:
docs = comments.usercomment.as_matrix()

In [ ]:
bagofwords = count.fit_transform(docs)

In [ ]:
count = Counter()
print(count.most_common)

In [ ]:
tf = tfidf.fit_transform(count.fit_transform(docs)).toarray()

In [ ]:
bagofwords = count.fit_transform(docs_clean)

## PCA

In [52]:
from sklearn import decomposition

In [ ]:
X = comments.tokens_stemmed

pca = decomposition.PCA(n_components=10)

pca.fit(X)
X_trans = pca.transform(X)

a bit of cleaning - remove any comments where there is not comment text

In [172]:
comments['commentID'] = comments.index

# remove any comments with no comment text
comments = comments[pd.notnull(comments.usercomment)]

# replace NaN usernames with 'anon'
comments['username'].fillna('anon', inplace=True)

# sanity check
recipes.numberofcomments.sum(), len(comments), len(recipes)

(27776, 27775, 100)

### tokenize 

In [173]:
def make_lowercase(comment):
    return remove_newlines(comment.lower())

def tokenize(comment):
    comment = remove_newlines(comment)
    return [word for word in word_tokenize(remove_newlines(comment).lower()) if word not in mystops]
    
def tokenize_stem(comment):
    tokens = word_tokenize(remove_newlines(comment).lower())
    return [porter.stem(word) for word in tokens if word not in mystops]

def remove_newlines(comment):    
    return re.sub(r"\n", " ", comment)

tokenize, clean up

In [174]:
comments['usercomment'] = comments.usercomment.apply(remove_newlines)
comments['usercomment_lower'] = comments.usercomment.apply(make_lowercase)
comments['tokens'] = comments.usercomment.apply(tokenize)
comments['tokens_stemmed'] = comments.usercomment.apply(tokenize_stem)

### make new dataframe with sentences separated

In [175]:
def tokenize_sentences(comment):
    return nltk.sent_tokenize(remove_newlines(comment.lower()))
    
    
def make_sentences_frame(frame, identifier, paragraph, how='merge'):
    sentences = pd.concat([pd.Series(row[identifier], tokenize_sentences((row[paragraph]))) for _, row in frame.iterrows()]).reset_index()
    sentences.columns = ['sentence', identifier]
    
    if how == 'merge':
        return sentences.merge(frame, left_on=identifier, right_on=identifier, how='outer', sort=False, suffixes=('_l','_r')) 
    elif how == 'nomerge': 
        return sentences
    else: 
        return sentences.merge(frame, left_on=identifier, right_on=identifier, how='outer', sort=False, suffixes=('_l','_r')) 

# slightly faster version
def make_sentences_frame2(frame, identifier, paragraph, how='merge'):
    sentences = pd.DataFrame((tokenize_sentences(row[paragraph]) for _, row in frame.iterrows()), index=frame[identifier]).stack()
    sentences = sentences.reset_index()[[0, identifier]] # var1 variable is currently labeled 0
    sentences.columns = ['sentence', identifier] # renaming var1
    
    if how == 'merge':
        return sentences.merge(frame, left_on=identifier, right_on=identifier, how='outer', sort=False, suffixes=('_l','_r')) 
    elif how == 'nomerge': 
        return sentences
    else: 
        return sentences.merge(frame, left_on=identifier, right_on=identifier, how='outer', sort=False, suffixes=('_l','_r')) 


apply to data

In [176]:
comments_sentences = make_sentences_frame2(comments, 'commentID','usercomment',how='merge')
# comments

In [180]:
comments_sentences2 = make_sentences_frame(comments, 'commentID','usercomment',how='merge')
comments_sentences2.to_csv('comments_intermediateSKdata.csv')

In [12]:
commentslist = []
for comment in comments.tokens:
    commentslist += [' '.join(comment)]
#     commentslist += [comment.strip('\n').lower()]
#     cleaned = re.sub(r"\n", " ", comment.lower())  
#     commentslist += [re.sub(r"'", "", cleaned)]
    
# #     commentslist += [re.sub(r"\r\n", "  ", comment.lower())]
    
commentslist_joined = ' '.join(commentslist)
# commentslist_joined

"oh – little piece heaven right time holidays yes please ive never made sweet potato pie i’d love try holidays definitely going must make list north carolina sweet potatoes season year-round always looking new ways eat thinking morning would like make sweet potato pie thanksgiving recipe right time bet knew finally sweet potato pie isn’t like pumpkin pie gets cuter everyday ok one who’s disappointed aren’t baby pictures jump can’t wait try husband loves sweet potato pie i’m they’re always heavy lacking something can’t wait try one thanks always great inspirations might make could potentially seem like cheesecake people always baby pictures sometimes embedded sometimes linked off-site page become aware blog enticing recipe i’m big fan sweet potato pie wait try recipe pretty pictures beautiful blog oh love idea whipped egg whites lightening whole thing plus anything buttermilk really makes happy i’ll admit i’ve never sweet potato pie tuber never made childhood go dinner someone sweet pot

In [ ]:
comments[comments.usercomment.str.contains('suggest')]

In [ ]:
comments[comments.usercomment.str.contains('omitted')]

testing and timing 

testing sentence map